# Date Time Feature Engineering
## Pandas

https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.date.html

In [47]:
import pandas as pd

# Create a dictionary with dates
data = {'Date': ['2023-08-04', '2023-08-05', '2023-08-06'], 
        'Value': [100, 200, 300]}

# Create a pandas dataframe from the dictionary
df = pd.DataFrame.from_dict(data)

# Convert Date to Datetime format.
df['Date'] = pd.to_datetime(df.Date, format='%Y-%m-%d %H:%M:%S')

# Print the dataframe
df

,Date,Value
0,2023-08-04,100
1,2023-08-05,200
2,2023-08-06,300


In [48]:
# Feature Engineering
df.loc[:,'year'] = df['Date'].dt.year  
df['Week_Number'] = df['Date'].dt.isocalendar().week # ISO years always start on a Monday.
df.loc[:,'weekofyear'] = df['Date'].dt.strftime('%U') # Default starts on Sunday.
df.loc[:,'month'] = df['Date'].dt.month  
df.loc[:,'dayofweek'] = df['Date'].dt.dayofweek # Monday = 0 and Sunday = 6
df.loc[:, 'weekend'] = (df.Date.dt.weekday >=5).astype(int)  
df.loc[:, 'hour'] = df['Date'].dt.hour

# Print the dataframe
df

,Date,Value,year,Week_Number,weekofyear,month,dayofweek,weekend,hour
0,2023-08-04,100,2023,31,31,8,4,0,0
1,2023-08-05,200,2023,31,31,8,5,1,0
2,2023-08-06,300,2023,31,32,8,6,1,0


In [45]:
import pandas as pd  

#create a series of datetime with a frequency of 10 hours  
s = pd.date_range('2020-01-06', '2020-01-10', freq='10H').to_series()  

#create some features based on datetime  
features = {  
            "dayofweek": s.dt.dayofweek.values,  
            "dayofyear": s.dt.dayofyear.values,  
            "hour": s.dt.hour.values,  
            "is_leap_year": s.dt.is_leap_year.values,  
            "quarter": s.dt.quarter.values,  
            "weekofyear": s.dt.isocalendar().week  
            } 

# Create a pandas dataframe from the dictionary
df = pd.DataFrame.from_dict(features)

df

,dayofweek,dayofyear,hour,is_leap_year,quarter,weekofyear
2020-01-06 00:00:00,0,6,0,True,1,2
2020-01-06 10:00:00,0,6,10,True,1,2
2020-01-06 20:00:00,0,6,20,True,1,2
2020-01-07 06:00:00,1,7,6,True,1,2
2020-01-07 16:00:00,1,7,16,True,1,2
2020-01-08 02:00:00,2,8,2,True,1,2
2020-01-08 12:00:00,2,8,12,True,1,2
2020-01-08 22:00:00,2,8,22,True,1,2
2020-01-09 08:00:00,3,9,8,True,1,2
2020-01-09 18:00:00,3,9,18,True,1,2


In [69]:
# Create a dictionary with dates
data = {'date': ['2016-09-01','2017-04-01','2017-08-01','2017-12-01','2017-09-01','2016-09-01','2017-04-01','2017-08-01'], 
        'customer_id': [146361,180838,157857,159772,80014,157857,159772,80014],
        'cat1':[2,4,3,5,3,4,3,5],
        'cat2':[2,1,3,1,2,1,3,1],
        'cat3':[0,0,1,1,1,0,0,1],
        'num1':[-0.518679,0.415853,-2.061687,-0.276558,-1.456827,-0.518679,0.415853,-2.061687]}

# Create a pandas dataframe from the dictionary
df = pd.DataFrame.from_dict(data)

# Convert Date to Datetime format.
df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d %H:%M:%S')

# Print the dataframe
df

def generate_features(df):  
    # create a bunch of features using the date column  
    df.loc[:, 'year'] = df['date'].dt.year  
    df.loc[:, 'weekofyear'] = df['date'].dt.isocalendar().week 
    df.loc[:, 'month'] = df['date'].dt.month  
    df.loc[:, 'dayofweek'] = df['date'].dt.dayofweek  
    df.loc[:, 'weekend'] = (df['date'].dt.weekday >=5).astype(int)  
    
    # create an aggregate dictionary  
    aggs = {}  
    # for aggregation by month, we calculate the  
    # number of unique month values and also the mean  
    aggs['month'] = ['nunique', 'mean']  
    aggs['weekofyear'] = ['nunique', 'mean']  
    # we aggregate by num1 and calculate sum, max, min  
    # and mean values of this column  
    aggs['num1'] = ['sum','max','min','mean']  
    # for customer_id, we calculate the total count  
    aggs['customer_id'] = ['size']  
    # again for customer_id, we calculate the total unique  
    aggs['customer_id'] = ['nunique']  
    # we group by customer_id and calculate the aggregates  
    agg_df = df.groupby('customer_id').agg(aggs)  
    agg_df = agg_df.reset_index()  
    return agg_df

# Run Aggregation
agg_df = generate_features(df)

# Print and sort by column and aggregation metric.
agg_df.sort_values(by=('month','nunique'), ascending=False)

customer_id   month      weekofyear            num1                      \
              nunique mean    nunique  mean       sum       max       min   
0       80014       2  8.5          2  33.0 -3.518514 -1.456827 -2.061687   
2      157857       2  8.5          2  33.0 -2.580366 -0.518679 -2.061687   
3      159772       2  8.0          2  30.5  0.139295  0.415853 -0.276558   
1      146361       1  9.0          1  35.0 -0.518679 -0.518679 -0.518679   
4      180838       1  4.0          1  13.0  0.415853  0.415853  0.415853   

            customer_id  
       mean     nunique  
0 -1.759257           1  
2 -1.290183           1  
3  0.069647           1  
1 -0.518679           1  
4  0.415853           1